# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
import lxml

def nhl_correlation(): 
    nhl_df=pd.read_csv("assets/nhl.csv", skipfooter=136, engine='python')
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,8]]
    nhl_df = nhl_df.drop([0, 9, 18, 26])
    nhl_df['Win/Loss Ratio'] = nhl_df['W'].astype(int)/(nhl_df['W'].astype(int) + nhl_df['L'].astype(int))
    nhl_df = nhl_df.iloc[:,[0,-1]]
    nhl_df['Metropolitan area'] = nhl_df['team'].str.extract('(\w* \w*)(?=\**)', expand =False)
    new_metropolitan_names = {'Tampa Bay':'Tampa Bay Area',
                             'Boston Bruins':'Boston',
                             'Toronto Maple':'Toronto',
                             'Florida Panthers':'Miami–Fort Lauderdale',
                             'Detroit Red':'Detroit',
                             'Montreal Canadiens':'Montreal',
                             'Ottawa Senators':'Ottawa',
                             'Buffalo Sabres': 'Buffalo',
                             'Washington Capitals':'Washington, D.C.',
                             'Pittsburgh Penguins':'Pittsburgh',
                             'Philadelphia Flyers':'Philadelphia',
                             'Columbus Blue':'Columbus',
                             'New Jersey':'New York City',
                             'Carolina Hurricanes':'Raleigh',
                             'New York':'New York City',
                             'Nashville Predators':'Nashville',
                             'Winnipeg Jets':'Winnipeg',
                             'Minnesota Wild':'Minneapolis–Saint Paul',
                             'Colorado Avalanche':'Denver',
                             ' Louis':'St. Louis',
                             'Dallas Stars':'Dallas–Fort Worth',
                             'Chicago Blackhawks':'Chicago',
                             'Vegas Golden':'Las Vegas',
                             'Anaheim Ducks':'Los Angeles',
                             'San Jose':'San Francisco Bay Area',
                             'Calgary Flames':'Calgary',
                             'Edmonton Oilers':'Edmonton',
                             'Vancouver Canucks':'Vancouver',
                             'Arizona Coyotes':'Phoenix'}
    nhl_df['Metropolitan area'] = nhl_df['Metropolitan area'].replace(new_metropolitan_names)
    nhl_df = nhl_df.set_index('Metropolitan area')
    cities = cities.set_index('Metropolitan area')
    df_nhl = pd.merge(cities, nhl_df, how='inner', left_index=True, right_index=True)
    df_nhl['Population (2016 est.)[8]'] = df_nhl['Population (2016 est.)[8]'].astype(float)
    ratios = df_nhl.groupby('Metropolitan area').agg({'Win/Loss Ratio':np.mean,
                                                 'Population (2016 est.)[8]':np.median})    
    population_by_region = ratios['Population (2016 est.)[8]'].tolist()
    win_loss_by_region = ratios['Win/Loss Ratio'].tolist()

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]


In [3]:
display(nhl_correlation())

0.012486162921209907

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


def nba_correlation():
    nba_df=pd.read_csv("assets/nba.csv", skipfooter=132, engine='python')
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,7]]
    nba_df['Win/Loss Ratio'] = nba_df['W'].astype(int)/(nba_df['W'].astype(int) + nba_df['L'].astype(int))
    nba_df = nba_df.iloc[:,[0,-1]]
    nba_df['Metropolitan area'] = nba_df['team'].str.extract('(.*)(?=\ \w*\**)', expand =False)
    new_metropolitan_names = {'Indiana':'Indianapolis',
                             'Miami':'Miami–Fort Lauderdale',
                             'Washington':'Washington, D.C.',
                             'New York':'New York City',
                             'Brooklyn':'New York City',
                             'Golden State':'San Francisco Bay Area',
                             'Portland Trail':'Portland',
                             'Utah':'Salt Lake City',
                             'Minnesota':'Minneapolis–Saint Paul',
                             'Dallas':'Dallas–Fort Worth'}
    nba_df['Metropolitan area'] = nba_df['Metropolitan area'].replace(new_metropolitan_names)
    nba_df = nba_df.set_index('Metropolitan area')
    cities = cities.set_index('Metropolitan area')
    df_nba = pd.merge(cities, nba_df, how='inner', left_index=True, right_index=True)
    df_nba['Population (2016 est.)[8]'] = df_nba['Population (2016 est.)[8]'].astype(float)
    ratios = df_nba.groupby('Metropolitan area').agg({'Win/Loss Ratio':np.mean,
                                                 'Population (2016 est.)[8]':np.median})
    population_by_region = ratios['Population (2016 est.)[8]'].tolist()
    win_loss_by_region = ratios['Win/Loss Ratio'].tolist()
    

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]


In [5]:
display(nba_correlation())

-0.17657160252844623

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [6]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


def mlb_correlation(): 
    mlb_df=pd.read_csv("assets/mlb.csv", skipfooter=120, engine='python')
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,6]]
    mlb_df['Win/Loss Ratio'] = mlb_df['W'].astype(int)/(mlb_df['W'].astype(int) + mlb_df['L'].astype(int))
    mlb_df = mlb_df.iloc[:,[0,-1]]
    mlb_df['Metropolitan area'] = mlb_df['team'].str.extract('(.*)(?=\ \w*\**)', expand =False)
    new_metropolitan_names = {'Tampa Bay':'Tampa Bay Area',
                             'Boston Red':'Boston',
                             'Toronto Blue':'Toronto',
                             'Miami':'Miami–Fort Lauderdale',
                             'Baltimore Orioles':'Baltimore',
                             'Washington':'Washington, D.C.',
                             'New York':'New York City',
                             'Minnesota':'Minneapolis–Saint Paul',
                             'Texas':'Dallas–Fort Worth',
                             'Chicago White':'Chicago',
                             'Oakland':'San Francisco Bay Area',
                             'San Francisco':'San Francisco Bay Area',
                             'Arizona':'Phoenix',
                             'Colorado':'Denver'}
    mlb_df['Metropolitan area'] = mlb_df['Metropolitan area'].replace(new_metropolitan_names)
    mlb_df = mlb_df.set_index('Metropolitan area')
    cities = cities.set_index('Metropolitan area')
    df_mlb = pd.merge(cities, mlb_df, how='inner', left_index=True, right_index=True)
    df_mlb['Population (2016 est.)[8]'] = df_mlb['Population (2016 est.)[8]'].astype(float)
    ratios = df_mlb.groupby('Metropolitan area').agg({'Win/Loss Ratio':np.mean,
                                                 'Population (2016 est.)[8]':np.median})
    population_by_region = ratios['Population (2016 est.)[8]'].tolist()
    win_loss_by_region = ratios['Win/Loss Ratio'].tolist()
    stats.pearsonr(population_by_region, win_loss_by_region)[0]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [7]:
display(mlb_correlation())

0.15027698302669307

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [8]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


def nfl_correlation(): 
    nfl_df=pd.read_csv("assets/nfl.csv", skipfooter=160, engine='python')
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5]]
    nfl_df = nfl_df.drop([0, 5, 10, 15, 20, 25, 30, 35])
    nfl_df['Win/Loss Ratio'] = nfl_df['W'].astype(int)/(nfl_df['W'].astype(int) + nfl_df['L'].astype(int))
    nfl_df = nfl_df.iloc[:,[-3,-1]]
    nfl_df['Metropolitan area'] = nfl_df['team'].str.extract('(.*)(?=\ \w*\**)', expand =False)
    new_metropolitan_names = {'Tampa Bay':'Tampa Bay Area',
                             'New England':'Boston',
                             'Oakland':'San Francisco Bay Area',
                             'San Francisco':'San Francisco Bay Area',
                             'Miami':'Miami–Fort Lauderdale',
                             'Washington':'Washington, D.C.',
                             'Carolina':'Charlotte',
                             'New York':'New York City',
                             'Tennessee':'Nashville',
                             'Minnesota':'Minneapolis–Saint Paul',
                             'Dallas':'Dallas–Fort Worth',
                             'Arizona':'Phoenix'}
    nfl_df['Metropolitan area'] = nfl_df['Metropolitan area'].replace(new_metropolitan_names)
    nfl_df = nfl_df.set_index('Metropolitan area')
    cities = cities.set_index('Metropolitan area')
    df_nfl = pd.merge(cities, nfl_df, how='inner', left_index=True, right_index=True)
    df_nfl['Population (2016 est.)[8]'] = df_nfl['Population (2016 est.)[8]'].astype(float)
    ratios = df_nfl.groupby('Metropolitan area').agg({'Win/Loss Ratio':np.mean,
                                                 'Population (2016 est.)[8]':np.median})
    population_by_region = ratios['Population (2016 est.)[8]'].tolist()
    win_loss_by_region = ratios['Win/Loss Ratio'].tolist()

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [9]:
display(nfl_correlation())

0.004922112149349393

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [11]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re



def sports_team_performance():
    mlb_df=pd.read_csv("assets/mlb.csv", skipfooter=120, engine='python')
    nhl_df=pd.read_csv("assets/nhl.csv", skipfooter=136, engine='python')
    nba_df=pd.read_csv("assets/nba.csv", skipfooter=132, engine='python')
    nfl_df=pd.read_csv("assets/nfl.csv", skipfooter=160, engine='python')
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    nhl_df = nhl_df.drop([0, 9, 18, 26])
    nhl_df['Win/Loss Ratio'] = nhl_df['W'].astype(int)/(nhl_df['W'].astype(int) + nhl_df['L'].astype(int))
    nhl_df = nhl_df.iloc[:,[0,-1]]
    nhl_df['Metropolitan area'] = nhl_df['team'].str.extract('(\w* \w*)(?=\**)', expand =False)
    new_metropolitan_names = {'Tampa Bay':'Tampa Bay Area',
                             'Boston Bruins':'Boston',
                             'Toronto Maple':'Toronto',
                             'Florida Panthers':'Miami–Fort Lauderdale',
                             'Detroit Red':'Detroit',
                             'Montreal Canadiens':'Montreal',
                             'Ottawa Senators':'Ottawa',
                             'Buffalo Sabres': 'Buffalo',
                             'Washington Capitals':'Washington, D.C.',
                             'Pittsburgh Penguins':'Pittsburgh',
                             'Philadelphia Flyers':'Philadelphia',
                             'Columbus Blue':'Columbus',
                             'New Jersey':'New York City',
                             'Carolina Hurricanes':'Raleigh',
                             'New York':'New York City',
                             'Nashville Predators':'Nashville',
                             'Winnipeg Jets':'Winnipeg',
                             'Minnesota Wild':'Minneapolis–Saint Paul',
                             'Colorado Avalanche':'Denver',
                             ' Louis':'St. Louis',
                             'Dallas Stars':'Dallas–Fort Worth',
                             'Chicago Blackhawks':'Chicago',
                             'Vegas Golden':'Las Vegas',
                             'Anaheim Ducks':'Los Angeles',
                             'San Jose':'San Francisco Bay Area',
                             'Calgary Flames':'Calgary',
                             'Edmonton Oilers':'Edmonton',
                             'Vancouver Canucks':'Vancouver',
                             'Arizona Coyotes':'Phoenix'}
    nhl_df['Metropolitan area'] = nhl_df['Metropolitan area'].replace(new_metropolitan_names)
    nhl_df = nhl_df.set_index('Metropolitan area')
    nhl_df = nhl_df.drop(['team'], axis=1)

    nba_df['Win/Loss Ratio'] = nba_df['W'].astype(int)/(nba_df['W'].astype(int) + nba_df['L'].astype(int))
    nba_df = nba_df.iloc[:,[0,-1]]
    nba_df['Metropolitan area'] = nba_df['team'].str.extract('(.*)(?=\ \w*\**)', expand =False)
    new_metropolitan_names = {'Indiana':'Indianapolis',
                             'Miami':'Miami–Fort Lauderdale',
                             'Washington':'Washington, D.C.',
                             'New York':'New York City',
                             'Brooklyn':'New York City',
                             'Golden State':'San Francisco Bay Area',
                             'Portland Trail':'Portland',
                             'Utah':'Salt Lake City',
                             'Minnesota':'Minneapolis–Saint Paul',
                             'Dallas':'Dallas–Fort Worth'}
    nba_df['Metropolitan area'] = nba_df['Metropolitan area'].replace(new_metropolitan_names)
    nba_df = nba_df.set_index('Metropolitan area')
    nba_df = nba_df.drop(['team'], axis=1)

    mlb_df['Win/Loss Ratio'] = mlb_df['W'].astype(int)/(mlb_df['W'].astype(int) + mlb_df['L'].astype(int))
    mlb_df = mlb_df.iloc[:,[0,-1]]
    mlb_df['Metropolitan area'] = mlb_df['team'].str.extract('(.*)(?=\ \w*\**)', expand =False)
    new_metropolitan_names = {'Tampa Bay':'Tampa Bay Area',
                             'Boston Red':'Boston',
                             'Toronto Blue':'Toronto',
                             'Miami':'Miami–Fort Lauderdale',
                             'Baltimore Orioles':'Baltimore',
                             'Washington':'Washington, D.C.',
                             'New York':'New York City',
                             'Minnesota':'Minneapolis–Saint Paul',
                             'Texas':'Dallas–Fort Worth',
                             'Chicago White':'Chicago',
                             'Oakland':'San Francisco Bay Area',
                             'San Francisco':'San Francisco Bay Area',
                             'Arizona':'Phoenix',
                             'Colorado':'Denver'}
    mlb_df['Metropolitan area'] = mlb_df['Metropolitan area'].replace(new_metropolitan_names)
    mlb_df = mlb_df.set_index('Metropolitan area')
    mlb_df = mlb_df.drop(['team'], axis=1)
    
    nfl_df = nfl_df.drop([0, 5, 10, 15, 20, 25, 30, 35])
    nfl_df['Win/Loss Ratio'] = nfl_df['W'].astype(int)/(nfl_df['W'].astype(int) + nfl_df['L'].astype(int))
    nfl_df = nfl_df.iloc[:,[-3,-1]]
    nfl_df['Metropolitan area'] = nfl_df['team'].str.extract('(.*)(?=\ \w*\**)', expand =False)
    new_metropolitan_names = {'Tampa Bay':'Tampa Bay Area',
                             'New England':'Boston',
                             'Oakland':'San Francisco Bay Area',
                             'San Francisco':'San Francisco Bay Area',
                             'Miami':'Miami–Fort Lauderdale',
                             'Washington':'Washington, D.C.',
                             'Carolina':'Charlotte',
                             'New York':'New York City',
                             'Tennessee':'Nashville',
                             'Minnesota':'Minneapolis–Saint Paul',
                             'Dallas':'Dallas–Fort Worth',
                             'Arizona':'Phoenix'}
    nfl_df['Metropolitan area'] = nfl_df['Metropolitan area'].replace(new_metropolitan_names)
    nfl_df = nfl_df.set_index('Metropolitan area')
    nfl_df = nfl_df.drop(['team'], axis=1)

    nhl_mlb = pd.merge(nhl_df, mlb_df, how='inner', left_index=True, right_index=True)
    nhl_mlb = nhl_mlb.groupby('Metropolitan area').agg({'Win/Loss Ratio_x':np.mean,
                                                   'Win/Loss Ratio_y':np.mean})
    nhl_nfl = pd.merge(nhl_df, nfl_df, how='inner', left_index=True, right_index=True)
    nhl_nfl = nhl_nfl.groupby('Metropolitan area').agg({'Win/Loss Ratio_x':np.mean,
                                                   'Win/Loss Ratio_y':np.mean})
    nhl_nba = pd.merge(nhl_df, nba_df, how='inner', left_index=True, right_index=True)
    nhl_nba = nhl_nba.groupby('Metropolitan area').agg({'Win/Loss Ratio_x':np.mean,
                                                   'Win/Loss Ratio_y':np.mean})
    nba_mlb = pd.merge(nba_df, mlb_df, how='inner', left_index=True, right_index=True)
    nba_mlb = nba_mlb.groupby('Metropolitan area').agg({'Win/Loss Ratio_x':np.mean,
                                                   'Win/Loss Ratio_y':np.mean})
    nba_nfl = pd.merge(nba_df, nfl_df, how='inner', left_index=True, right_index=True)
    nba_nfl = nba_nfl.groupby('Metropolitan area').agg({'Win/Loss Ratio_x':np.mean,
                                                   'Win/Loss Ratio_y':np.mean})
    nfl_mlb = pd.merge(nfl_df, mlb_df, how='inner', left_index=True, right_index=True)
    nfl_mlb = nfl_mlb.groupby('Metropolitan area').agg({'Win/Loss Ratio_x':np.mean,
                                                   'Win/Loss Ratio_y':np.mean})


    NHL_NBA = stats.ttest_rel(nhl_nba['Win/Loss Ratio_x'], nhl_nba['Win/Loss Ratio_y'])

    NHL_MLB = stats.ttest_rel(nhl_mlb['Win/Loss Ratio_x'], nhl_mlb['Win/Loss Ratio_y'])

    NHL_NFL = stats.ttest_rel(nhl_nfl['Win/Loss Ratio_x'], nhl_nfl['Win/Loss Ratio_y'])

    NBA_MLB = stats.ttest_rel(nba_mlb['Win/Loss Ratio_x'], nba_mlb['Win/Loss Ratio_y'])

    NBA_NFL = stats.ttest_rel(nba_nfl['Win/Loss Ratio_x'], nba_nfl['Win/Loss Ratio_y'])

    NFL_MLB = stats.ttest_rel(nfl_mlb['Win/Loss Ratio_x'], nfl_mlb['Win/Loss Ratio_y'])

    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    p_values.loc['NBA', 'NHL'] = NHL_NBA[1]
    p_values.loc['NHL', 'NBA'] = NHL_NBA[1]
    p_values.loc['NHL', 'MLB'] = NHL_MLB[1]
    p_values.loc['MLB', 'NHL'] = NHL_MLB[1]
    p_values.loc['NFL', 'NHL'] = NHL_NFL[1]
    p_values.loc['NHL', 'NFL'] = NHL_NFL[1]
    p_values.loc['NBA', 'MLB'] = NBA_MLB[1]
    p_values.loc['MLB', 'NBA'] = NBA_MLB[1]
    p_values.loc['NBA', 'NFL'] = NBA_NFL[1]
    p_values.loc['NFL', 'NBA'] = NBA_NFL[1]
    p_values.loc['NFL', 'MLB'] = NFL_MLB[1]
    p_values.loc['MLB', 'NFL'] = NFL_MLB[1]
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values
sports_team_performance()

,NFL,NBA,NHL,MLB
NFL,NaN,0.941792,0.030883,0.802069
NBA,0.941792,NaN,0.022297,0.950540
NHL,0.030883,0.022297,NaN,0.000708
MLB,0.802069,0.950540,0.000708,NaN
